# PPA3 Database Table Viewer
If PPA is live and people are using it, and you want to view tables, please do so through this tool rather than Arc Pro. Viewing in Pro will create a schema lock on the tables, so when a user runs the tool, it will cause an error because it cannot log the run output to the locked database tables.

This viewer gets around this issue by creating an in-memory copy of desired gdb tables.

In [10]:
import pandas as pd

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

file_gdb = r'\\arcserver-svr\D\PPA3_SVR\PPA3_GIS_SVR\PPA3_run_data.gdb'
arcpy.env.workspace = file_gdb

fc_pmaster = 'project_master'
f_uid = 'project_uid'

df_pmaster = pd.DataFrame.spatial.from_featureclass(fc_pmaster)

def esri_object_to_df(in_esri_obj, esri_obj_fields, index_field=None):
    '''converts esri gdb table, feature class, feature layer, or SHP to pandas dataframe'''
    data_rows = []
    with arcpy.da.SearchCursor(in_esri_obj, esri_obj_fields) as cur:
        for row in cur:
            out_row = list(row)
            data_rows.append(out_row)

    out_df = pd.DataFrame(data_rows, index=index_field, columns=esri_obj_fields)
    return out_df

print([t for t in arcpy.ListTables()])


['rp_artexp_vmt', 'rp_artexp_econ', 'rp_artexp_eq', 'rp_artexp_mm', 'rp_artexp_sgr', 'rp_fwy_vmt', 'rp_fwy_cong', 'rp_fwy_mm', 'rp_fwy_econ', 'rp_fwy_frgt', 'rp_fwy_saf', 'rp_artsgr_sgr', 'cd_compactdev', 'cd_mixeduse', 'cd_houschoice', 'cd_naturpres', 'rp_artexp_cong', 'rp_artexp_frgt', 'rp_artexp_saf', 'cd_trnchoice', 'cd_existgasset', 'TEST_TABLE']


## View of most recent entries into master table

In [11]:
df_pmaster.sort_values(by='time_created', ascending=False).head()

,OBJECTID,project_uid,proj_name,juris,proj_type,aadt,pci,posted_speed,len_mi,comm_type,time_created,user_email,perf_outcomes,for_review,SHAPE
245,539,ae44b9cf-8bcb-4316-80b6-76cc36fba74b,Yolo Causeway Segment 3,Caltrans D3,Freeway Expansion,163000.0,70.0,65.0,3.051947,Established Communities,2023-01-20 15:18:02,gballard-rosa@sacog.org,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13532326.1383, 4661077.088600002..."
244,538,dc6f1f35-648b-46f4-aed0-f438af0d99f0,Yolo 80 Segment 2,Caltrans D3,Freeway Expansion,117000.0,70.0,65.0,2.497230,Established Communities,2023-01-20 15:13:15,gballard-rosa@sacog.org,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13532344.099, 4661148.931199998]..."
243,537,dbce3f3d-ba7f-4cab-b323-dd4c2a33f7b4,Yolo 80 Segment 1,Caltrans D3,Freeway Expansion,157000.0,70.0,65.0,9.947685,Ag,2023-01-20 15:12:13,gballard-rosa@sacog.org,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13551867.3387, 4655670.929300003..."
242,536,44e1b622-0202-431a-8cf7-2dd018e6e00a,Yolo 80 Managed Lanes Full Segment,Caltrans D3,Freeway Expansion,150400.0,70.0,65.0,15.544632,Established Communities,2023-01-20 15:10:57,gballard-rosa@sacog.org,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13532362.0596, 4661059.127999999..."
241,535,fe66fb95-c2ba-406c-b9d1-8251dfe9d8f6,I 80 Managed Lanes,Caltrans D3,Freeway Expansion,120000.0,0.0,65.0,16.102524,Established Communities,2023-01-20 15:01:25,gballard-rosa@sacog.org,Reduce VMT; Reduce Congestion; Encourage Multi...,NaN,"{""paths"": [[[-13532344.0989, 4661077.088600002..."


## Find UID values that are in any of the data tables but not in the master table

In [3]:


def get_uids(in_tbl):
    uids = []

    try:
        with arcpy.da.SearchCursor(in_tbl, field_names=[f_uid]) as cur:
            for row in cur:
                uids.append(row[0])
    except:
        print(f"{in_tbl} does not have field {f_uid}. Skipping...")
        
    return uids

data_tables = [t for t in arcpy.ListTables()]
pmaster_uids = df_pmaster[f_uid].to_list()

out_dict = {}

fake_id = 'UID_NOT_FOUND' # placeholder ID used when running tool locally for testing

for t in data_tables:
    tbl_uids = get_uids(t)
    
    uids_not_in_master = []
    for uid in tbl_uids:
        if uid not in pmaster_uids and uid != fake_id:
            uids_not_in_master.append(uid)
            
    out_dict[t] = uids_not_in_master
    if len(out_dict[t]) > 0:
        print(f"table {t} has the follwing UIDs not in the project_master table: {out_dict[t]}")
        
total_missing = sum([len(v) for tbl, v in out_dict.items()])
if total_missing == 0: print("no IDs found in subreport tables that are not also in project_master")
    

TEST_TABLE does not have field project_uid. Skipping...
no IDs found in subreport tables that are not also in project_master


## Show specific subreport data and for specific projects

In [13]:

subrpt_tbl = 'rp_fwy_econ'
project_uid = None

fields = [f.name for f in arcpy.ListFields(subrpt_tbl)]

#uncomment if you want to filter which fields appear
# fields = [f_uid, 'crash_cnt', 'crash_100mvmt'] # [f.name for f in arcpy.ListFields(subrpt_tbl)]

df_master_fields = [f_uid, 'time_created', 'user_email', 'len_mi', 'aadt']
df_subrpt = esri_object_to_df(subrpt_tbl, esri_obj_fields=fields, index_field=None)


df_jn = df_pmaster[df_master_fields].merge(df_subrpt, on=f_uid, suffixes=('','_y')) \
    .sort_values(by='time_created', ascending=False)
df_jn.head(10)

,project_uid,time_created,user_email,len_mi,aadt,OBJECTID,acc_drive_alljob,acc_drive_edu
56,ae44b9cf-8bcb-4316-80b6-76cc36fba74b,2023-01-20 15:18:02.000000,gballard-rosa@sacog.org,3.051947,163000.0,100,713744.3125,729.444458
55,dc6f1f35-648b-46f4-aed0-f438af0d99f0,2023-01-20 15:13:15.000000,gballard-rosa@sacog.org,2.497230,117000.0,99,708543.5625,731.922913
54,dbce3f3d-ba7f-4cab-b323-dd4c2a33f7b4,2023-01-20 15:12:13.000000,gballard-rosa@sacog.org,9.947685,157000.0,98,554717.1875,518.484619
53,44e1b622-0202-431a-8cf7-2dd018e6e00a,2023-01-20 15:10:57.000000,gballard-rosa@sacog.org,15.544632,150400.0,97,607230.5625,589.432068
52,fe66fb95-c2ba-406c-b9d1-8251dfe9d8f6,2023-01-20 15:01:25.000000,gballard-rosa@sacog.org,16.102524,120000.0,96,610871.7500,594.979797
51,2ca1575a-9c46-4623-a774-9ae6dcc8b3cd,2023-01-20 14:48:31.000000,gballard-rosa@sacog.org,17.207058,120000.0,95,628753.1875,620.102539
50,cc2867ef-29c0-4a33-bd1d-45a8c3552241,2023-01-19 22:43:17.000000,sathish_prakash@dot.ca.gov,9.445633,156960.0,94,526881.2500,479.817322
49,e1976726-a851-4ee5-904f-80c0392fbe41,2023-01-19 22:33:39.000001,sathish_prakash@dot.ca.gov,9.454070,156960.0,93,526881.2500,479.817322
48,1996cd29-85c1-4f2c-902f-f79542b31c51,2023-01-17 12:48:50.000000,nima.kabirinassab@dot.ca.gov,13.304194,181201.0,92,589030.1875,562.829773
47,fd2aa6db-d878-4f0e-8cb0-a63d1f4788a4,2023-01-17 12:43:38.000000,nima.kabirinassab@dot.ca.gov,13.052722,168670.0,91,583544.3750,559.334839


In [42]:
df_jn.loc[df_jn.crash_100mvmt > 100].head()

,project_uid,time_created,user_email,len_mi,aadt,crash_cnt,crash_100mvmt
21,55160fbd-9617-4c09-a05e-da8819a2b71c,2022-12-06 13:47:37,jhong@sacog.org,3.670994,5.0,136,463089.843750
45,fbc41452-2ed7-44ce-b1d1-3563bad0f73c,2023-01-17 06:06:46,rbissegger@markthomas.com,1.861210,58600.0,36,130.566345
